In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa as lr
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path
from scipy.ndimage import uniform_filter1d
import IPython.display as ipd

# Import samples
kick_samples = [lr.load(p)[0] for p in Path().glob('../data/Kicks/*.wav')]
sr = [lr.load(p)[1] for p in Path().glob('../data/Kicks/*.wav')]

def extract_features(signal,sr):
    return [
        # np.mean(lr.feature.zero_crossing_rate(signal)[0]),
        lr.feature.spectral_centroid(y=signal)[0,0],
        # lr.feature.spectral_bandwidth(y=signal)[0,0]
        # lr.feature.spectral_contrast(y=signal)[0,0],
        # lr.feature.spectral_flatness(y=signal)[0,0]
        # attack_time(signal),
        decay_time(signal)
    ]

def attack_time(signal):
    rms = lr.feature.rms(y=signal, frame_length=856, hop_length=40)
    times = lr.frames_to_time(np.arange(len(rms[0])), hop_length=40)
    rms = uniform_filter1d(rms, size=5)
    peak_index = np.argmax(rms[0][:1000])
    return times[peak_index]
    

def decay_time(signal):
    rms = lr.feature.rms(y=signal, frame_length=856, hop_length=40)
    times = lr.frames_to_time(np.arange(len(rms[0])), hop_length=40)
    rms = uniform_filter1d(rms, size=5)
    peak_index = np.argmax(rms)
    decay_threshold = rms[0][peak_index] * 0.1
    decay_index = peak_index + np.argmax(rms[0][peak_index:] < decay_threshold)
    decay_time = times[decay_index] - times[peak_index]
    return decay_time

plt.figure(figsize=(5,14))

kick_features = np.array([extract_features(x,sr) for x in kick_samples])

scaler = MinMaxScaler((0,1))
scaled_features = scaler.fit_transform(kick_features[:,:])

# weights = np.array([1,0.2,0.8])
weighted_features = scaled_features * 1

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns

# Apply K-means clustering
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(scaled_features)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=scaled_features[:, 0], y=scaled_features[:, 1], hue=labels, palette='viridis', s=100)
plt.xlabel('Feature 1 (e.g., Zero Crossing Rate)')
plt.ylabel('Feature 2 (e.g., Spectral Centroid)')
plt.title('K-means Clustering of Kick Samples')
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances

selected_sample_index = 7

# Calculate similarity within the cluster
similarity_scores = pairwise_distances(weighted_features)
sorted_indices = np.argsort(similarity_scores[selected_sample_index])  # Sort by highest similarity

sample_choices = [23,24,42,29,35]
# sample_choices[0]
sorted_indices

In [ ]:
sample = kick_samples[116]
ipd.Audio(sample,rate=sr[0])

In [ ]:
plt.figure(figsize=(14,5))
lr.display.waveshow(sample,sr=sr[0])

In [ ]:
plt.figure(figsize=(14,5))
rms = lr.feature.rms(y=sample, frame_length=856, hop_length=30)
rms = uniform_filter1d(rms, size=20)
times = lr.frames_to_time(np.arange(len(rms[0])),sr=sr[0],hop_length=48)
plt.plot(times,rms[0])
plt.xlabel('Time (s)')
plt.ylabel('RMS Energy')
plt.title('RMS Energy Envelope')
plt.legend()
plt.show()
